In [46]:
import cv2
import numpy as np
import json
from PIL import Image
import random
import os
import imutils


def resize_image(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return resized

def adjust_brightness(image, brightness):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = np.where((255 - v) < brightness, 255, v + brightness)
    final_hsv = cv2.merge((h, s, v))
    brighter = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return brighter

def rotate_image(image, angle):
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    rotated = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated


def adjust_bbox(bbox, scale_percent, angle, image_shape):
    x, y, w, h = bbox
    # center_x = x + w / 2
    # center_y = y + h / 2
    center_x = x + w / 1.7
    center_y = y + h / 1.7
    scale = scale_percent / 100
    new_w = w * scale
    new_h = h * scale
    new_center_x = center_x * scale
    new_center_y = center_y * scale
    rotation_matrix = cv2.getRotationMatrix2D((image_shape[1]/2, image_shape[0]/2), angle, 1)
    new_center_x, new_center_y = np.dot(rotation_matrix, np.array([new_center_x, new_center_y, 1]))
    x = int(new_center_x - new_w / 2)
    y = int(new_center_y - new_h / 2)
    w = int(new_w)
    h = int(new_h)
    return [x, y, w, h]




In [47]:

# Step 1
with open('cat_dog_annotations.json') as f:
    data = json.load(f)

image_metadata = data['_via_img_metadata']
new_annotations = {}

for key, value in image_metadata.items():
    filename = value['filename']
    size = value['size']
    regions = value['regions']

    # Step 2
    image = Image.open(filename)

    # Step 3
    resize_percent = random.randint(60, 100)
    resized_image = cv2.resize(np.array(image), (0, 0), fx=resize_percent/100, fy=resize_percent/100, interpolation=cv2.INTER_AREA)

    # Adjust bounding box coordinates for resized image
    for region in regions:
        shape_attributes = region['shape_attributes']
        bbox = [shape_attributes['x'], shape_attributes['y'], shape_attributes['width'], shape_attributes['height']]
        adjusted_bbox = [int(b * resize_percent / 100) for b in bbox]

        region['shape_attributes']['x'] = adjusted_bbox[0]
        region['shape_attributes']['y'] = adjusted_bbox[1]
        region['shape_attributes']['width'] = adjusted_bbox[2]
        region['shape_attributes']['height'] = adjusted_bbox[3]

    # Step 4
    brightness_percent = random.randint(-50, 50)
    brighter_image = cv2.addWeighted(resized_image, 1, resized_image, 0, brightness_percent)

    # Steps 5 to 7 for each angle
    for angle in range(0, 360, 5):
        # Step 5
        rotated_image = rotate_image(brighter_image, angle)

        # Step 6
        # Adjust bounding box coordinates for rotated image
        new_regions = []
        for region in regions:
            shape_attributes = region['shape_attributes']
            bbox = [shape_attributes['x'], shape_attributes['y'], shape_attributes['width'], shape_attributes['height']]
            adjusted_bbox = adjust_bbox(bbox, resize_percent, angle, rotated_image.shape)
            # Draw a rectangle around the object
            x, y, w, h = adjusted_bbox
            cv2.rectangle(rotated_image, (x, y), (x+w, y+h), (0, 0, 255), thickness=2)

            new_region = {
                "shape_attributes": {
                    "name": "rect",
                    "x": adjusted_bbox[0],
                    "y": adjusted_bbox[1],
                    "width": adjusted_bbox[2],
                    "height": adjusted_bbox[3],
                    "bbox": bbox
                },
                "region_attributes": region["region_attributes"]
            }
            new_regions.append(new_region)


        # Save the new annotations
        base_name = os.path.splitext(filename)[0]
        new_key = f"{base_name}_{angle}"
        # new_key = f"{filename}_{angle}"
        new_annotations[new_key] = {
            "filename": f"{new_key}.png",
            "size": size,
            "regions": new_regions,
            "file_attributes": {}
        }

        # Step 7
        # Save the resulting image
        result_image = Image.fromarray(rotated_image)
        result_image.save(f"{new_key}.jpg")

# Step 8
# Define the new folder path
new_folder_path = 'folderr'

# Create the new folder if it doesn't exist
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

# Move all generated images to the new folder
for filename in os.listdir():
    if filename.endswith(".jpg"):
        os.rename(filename, os.path.join(new_folder_path, filename))

# Step 9
with open('new_annotations.json', 'w') as f:
    json.dump({"_via_img_metadata": new_annotations}, f)